In [13]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from stock2 import getAndProcessFinancialDataJSON, getStockTickers
from stock3 import getAndProcessMultiFinancialDataJSON
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [88]:
# Get stock data from the internet

# Get 20 random tickers from NYSE
tickers = getStockTickers(20)
#tickers = ["AMD", "GME", "AMC", "TSLA", "ZM", "FLO", "SQ", "GPS", "VTRS", "LNC", "NRG"]
tickers = ["AMD"]
# get stock data from those tickers
X_raw,y_raw = getAndProcessMultiFinancialDataJSON(tickers,look_fwd=2, interval="2m", rge="1mo")

#X_raw,y_raw = getAndProcessFinancialDataJSON("AMD",look_fwd=2)
#unix_time = getAndProcessFinancialDataJSON(tickers[0],justTimes=True)
print(tickers)
print("Data size: ",len(X_raw))

Loading data using URL [ https://query1.finance.yahoo.com/v8/finance/chart/AMD?interval=2m&range=1mo ]
['AMD']
Data size:  4288


In [89]:
X_raw_numpy = X_raw.to_numpy()
y_raw_numpy = y_raw.to_numpy()

In [90]:
# clean data
delete_list = []
for i in range(len(X_raw_numpy)):
    if(y_raw_numpy[i] == None):
        delete_list.append(i)
    for j in range(len(X_raw_numpy[0])):
        if(X_raw_numpy[i][j] == None):
            delete_list.append(i)
X_clean = np.delete(X_raw_numpy, delete_list, 0)
y_clean = np.delete(y_raw_numpy, delete_list, 0)
# np.isnan(y_raw[i])

print("original size:",len(y_raw))
print("elements to delete:",len(delete_list))
print("resulting size:",len(y_clean))
print("result+delete size:",len(y_clean) + len(delete_list))

original size: 4288
elements to delete: 0
resulting size: 4288
result+delete size: 4288


In [91]:
print(X_clean)
print(y_clean)
print(len(X_clean))
print(len(y_clean))

[['83.1238021850586' '82.9000015258789']
 ['82.9000015258789' '82.99649810791016']
 ['82.99649810791016' '82.77999877929688']
 ...
 ['78.74500274658203' '78.75499725341797']
 ['78.75499725341797' '78.77999877929688']
 ['78.77999877929688' '78.78500366210938']]
[['82.99649810791016']
 ['82.77999877929688']
 ['82.95999908447266']
 ...
 ['78.77999877929688']
 ['78.78500366210938']
 ['78.80000305175781']]
4288
4288


In [92]:
# Split into train, validate, and test sets
size = len(X_clean)

# Get test size and data
total_train_size = (size*8)//10
test_size = size - total_train_size

X_total_train_unnorm = X_clean[:total_train_size]
y_total_train_unnorm = y_clean[:total_train_size]

X_test_unnorm = X_clean[total_train_size:]
y_test_unnorm = y_clean[total_train_size:]

# Get train and validation size and data
train_size = (total_train_size*8)//10
val_size = total_train_size - train_size

X_train_unnorm = X_total_train_unnorm[:train_size]
y_train_unnorm = y_total_train_unnorm[:train_size]

X_val_unnorm = X_total_train_unnorm[train_size:]
y_val_unnorm = y_total_train_unnorm[train_size:]



In [93]:
print(total_train_size)
print(train_size)
print(val_size)
print(test_size)
print()
print(len(X_total_train_unnorm))
print(len(X_train_unnorm))
print(len(X_val_unnorm))
print(len(X_test_unnorm))
print()
print(len(y_total_train_unnorm))
print(len(y_train_unnorm))
print(len(y_val_unnorm))
print(len(y_test_unnorm))

3430
2744
686
858

3430
2744
686
858

3430
2744
686
858


In [162]:
# Scale between 0 and 1, best for LSTM models
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_norm = scaler.fit_transform(X_train_unnorm)
y_train_norm = scaler.fit_transform(y_train_unnorm)

X_val_norm = scaler.fit_transform(X_val_unnorm)
y_val_norm = scaler.fit_transform(y_val_unnorm)

X_test_norm = scaler.fit_transform(X_test_unnorm)
y_test_norm = scaler.fit_transform(y_test_unnorm)

In [177]:
# LSTM also needs to adhere to this shape
X =      np.reshape(X_train_norm, (     X_train_norm.shape[0],  -1, X_train_norm.shape[1]))
X_val =  np.reshape(X_val_norm,   (     X_val_norm.shape[0],    -1, X_val_norm.shape[1]))
X_test = np.reshape(X_test_norm,  (     X_test_norm.shape[0],   -1, X_test_norm.shape[1]))
y =      np.reshape(y_train_norm, (     y_train_norm.shape[0],  -1, y_train_norm.shape[1]))  # y_train_norm  
y_val =  np.reshape(y_val_norm,   (     y_val_norm.shape[0],    -1, y_val_norm.shape[1]))    # y_val_norm
y_test = np.reshape(y_test_norm,  (     y_test_norm.shape[0],   -1, y_test_norm.shape[1]))   # y_test_norm  

In [178]:
# Prints for sanity
print(X[0])
print(y[0])


print(len(X))
print(len(y))


[[0.47683322 0.45598547]]
[[0.46497444]]
2744
2744


In [179]:
# create the LSTM model
input_shape = (X.shape[0],  2)
print(input_shape)
model = Sequential()

# Shape = [# of elements, data per element (2)]
model.add(LSTM(4, input_shape=input_shape, return_sequences=True)) #, return_sequences=True))
model.add(LSTM(4, return_sequences=True))
model.add(LSTM(4))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=["accuracy"])

model.summary()

(2744, 2)
Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_61 (LSTM)               (None, 2744, 4)           112       
_________________________________________________________________
lstm_62 (LSTM)               (None, 2744, 4)           144       
_________________________________________________________________
lstm_63 (LSTM)               (None, 4)                 144       
_________________________________________________________________
dense_54 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 65        
Total params: 785
Trainable params: 785
Non-trainable params: 0
_________________________________________________________________


In [180]:
# Train
history = model.fit(X, y, epochs=100, batch_size=32, verbose=2, validation_data=(X_val, y_val))
print("Done!")

Epoch 1/100
86/86 - 3s - loss: 0.0716 - accuracy: 3.6443e-04 - val_loss: 0.0930 - val_accuracy: 0.0015
Epoch 2/100
86/86 - 0s - loss: 0.0378 - accuracy: 3.6443e-04 - val_loss: 0.0818 - val_accuracy: 0.0015
Epoch 3/100
86/86 - 0s - loss: 0.0178 - accuracy: 3.6443e-04 - val_loss: 9.9283e-04 - val_accuracy: 0.0029
Epoch 4/100
86/86 - 0s - loss: 2.9480e-04 - accuracy: 7.2886e-04 - val_loss: 5.7569e-04 - val_accuracy: 0.0029
Epoch 5/100
86/86 - 0s - loss: 2.3044e-04 - accuracy: 7.2886e-04 - val_loss: 7.2258e-04 - val_accuracy: 0.0029
Epoch 6/100
86/86 - 0s - loss: 2.1004e-04 - accuracy: 7.2886e-04 - val_loss: 6.2715e-04 - val_accuracy: 0.0029
Epoch 7/100
86/86 - 0s - loss: 2.0495e-04 - accuracy: 7.2886e-04 - val_loss: 5.3987e-04 - val_accuracy: 0.0029
Epoch 8/100
86/86 - 0s - loss: 2.0492e-04 - accuracy: 7.2886e-04 - val_loss: 5.4618e-04 - val_accuracy: 0.0029
Epoch 9/100
86/86 - 0s - loss: 2.0090e-04 - accuracy: 7.2886e-04 - val_loss: 5.6178e-04 - val_accuracy: 0.0029
Epoch 10/100
86/86 - 

In [ ]:
# plot loss
# plot accuracy